# Train on notebook

In [ ]:
# Basic libraries
import torch
import torch.nn as n
import torch.nn.functional as F
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from PIL import Image
from typing import List
import torch.optim as optim
from tqdm import tqdm
from omegaconf import OmegaConf

In [ ]:
os.chdir("../")
# Personalized modules
from preprocess.dataloader import AntibodiesTree
from preprocess.patches_utils import PatchesUtilities

from osgen.embeddings import StyleExtractor
from osgen.utils import Utilities
from osgen.vae import VanillaVAE,VanillaEncoder, VanillaDecoder
from osgen.base import BaseModel
from osgen.nn import *
from osgen.unet import *
from osgen.loss import *
from osgen.pipeline import *

In [ ]:
# Load config
cfg = OmegaConf.load("/root/BiOSGen/configs/train_config.yml")
print(cfg.verbose)

In [ ]:
# Directory where your tumor PNGs are stored
directory = '/root/BiOSGen/data/NKX3'

for filename in os.listdir(directory):
    if filename.endswith('.png') and 'NKX3' not in filename and 'NKX' in filename:
        new_filename = filename.replace('NKX', 'NKX3')
        old_path = os.path.join(directory, filename)
        new_path = os.path.join(directory, new_filename)
        os.rename(old_path, new_path)
        print(f'Renamed: {filename} -> {new_filename}')


In [ ]:
# SRC antibodies
tree_src = AntibodiesTree(
    image_dir = "/root/BiOSGen/data/HE",
    mask_dir = "/root/BiOSGen/data/tissue_masks/HE",
    npz_dir = "/root/BiOSGen/data/bbox_info/HE_NKX3/HE"
)

# DST antibodies
tree_dst = AntibodiesTree(
    image_dir = "/root/BiOSGen/data/NKX3",
    mask_dir = "/root/BiOSGen/data/tissue_masks/NKX3",
    npz_dir = "/root/BiOSGen/data/bbox_info/HE_NKX3/NKX3"
)

# Print
print("Nb antibodies: ", tree_src.get_nb_antibodies())
print("Nb antibodies: ", tree_dst.get_nb_antibodies())

In [ ]:
torch.randint(0, len(tree_src.antibodies), (1,)).item()


In [ ]:
idx_src = 48 # torch.randint(0, len(tree_src.antibodies), (1,)).item()
patches_src = PatchesUtilities.get_image_patches(
    image = np.array(Image.open(tree_src.antibodies[idx_src])),
    tissue_mask = PatchesUtilities.get_tissue_mask(np.array(Image.open(tree_src.antibodies[idx_src]))),
)

idx_dst = 60 # torch.randint(0, len(tree_dst.antibodies), (1,)).item()
patches_dst = PatchesUtilities.get_image_patches(
    image = np.array(Image.open(tree_dst.antibodies[idx_dst])),
    tissue_mask = PatchesUtilities.get_tissue_mask(np.array(Image.open(tree_dst.antibodies[idx_dst]))),
)
print("idx_src: ", idx_src)
print("idx_dst: ", idx_dst)

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(20, 15))  # 1x5 grid

# Plot original source image
axes[0].imshow(np.array(Image.open(tree_src.antibodies[idx_src])), cmap='viridis')
axes[0].set_title(f'Original Tumor {idx_src}', fontsize=16)

# Plot original destination image
axes[1].imshow(np.array(Image.open(tree_dst.antibodies[idx_dst])), cmap='viridis')
axes[1].set_title(f'Style Tumor {idx_dst}', fontsize=16)

## DataLoader

In [ ]:
# src_l = []
# dst_l = []
# batch_size = 25  # Define the batch size

# for i in range(25): 
#     src_l.append(np.array(Image.open(tree_src.antibodies[i])))
#     dst_l.append(np.array(Image.open(tree_dst.antibodies[i])))

# TRAIN

In [ ]:
pipeline = OSGenPipeline()

In [ ]:
trainable_params = sum(p.numel() for p in pipeline.parameters() if p.requires_grad)
print(f"Number of trainable parameters: {trainable_params}")


In [ ]:
count = 0
count_float32 = 0
for name, module in pipeline.named_modules():
    for param_name, param in module.named_parameters(recurse=False):
        # print(f"{name}.{param_name}: dtype={param.dtype}")
        if param.dtype == torch.float32:
            count_float32 += 1
        count += 1

print(f"float32 percentage: {count_float32 / count * 100}%")


In [ ]:
# Hyperparameters
verbose = cfg.verbose
num_epochs = cfg.num_epochs
batch_size = cfg.batch_size
lr = cfg.lr
optimizer = optim.AdamW([p for p in pipeline.parameters() if p.requires_grad], 
                        lr=cfg.optimizer.params.lr, weight_decay=cfg.optimizer.params.weight_decay, eps=cfg.optimizer.params.eps)

channels = cfg.channels
height = cfg.height
width = cfg.width
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

lambda_content = cfg.lambda_content
lambda_style = cfg.lambda_style

# Training loop
# Put on training mode
pipeline.train()
requires_grad = True

# Early stopping parameters
early_stopping_patience = cfg.early_stopping_patience
epochs_without_improvement = 0
best_loss = float("inf")

# Save losses
losses = []
content_losses = []
style_losses = []

# Switch to main training with bfloat16
for epoch in tqdm(range(num_epochs)):
    # print(f"Epoch {epoch+1}/{num_epochs}")
    avg_loss = []
    content_loss = []
    style_loss = []

    for i, (src, dst) in enumerate(zip(patches_src, patches_dst)):
        # Convert to tensors
        src_tensor = Utilities.convert_numpy_to_tensor(src).to(device)
        dst_tensor = Utilities.convert_numpy_to_tensor(dst).to(device)

        timesteps = torch.randint(0, 1000, (batch_size,), device=device)  # Random timesteps

        # forward pass
        decoded = pipeline(src_tensor, dst_tensor, timesteps)

        # Handle gradients issues
        requires_grad = decoded.requires_grad
        # print(requires_grad)
        if not requires_grad:
            break

        # Compute loss
        content_l, style_l, total_loss = pipeline.compute_loss(src_tensor, dst_tensor, decoded, lambda_content, lambda_style)

        # Break if total_loss is NaN
        if torch.isnan(total_loss):
            print(f"NaN loss at epoch {epoch+1}, batch {i+1}")
            break

        print(f"Epoch {epoch+1}/{num_epochs}, Batch {i+1}/{len(patches_src)}, "
                f"Content Loss: {content_l.item():.4f}, Style Loss: {style_l.item():.4f}, "
                f"Total Loss: {total_loss.item():.4f}")

        avg_loss.append(total_loss.item())
        content_loss.append(content_l.item())
        style_loss.append(style_l.item())

        # Backpropagation
        optimizer.zero_grad()
        total_loss.backward()
        optimizer.step()


    current_loss = np.mean(avg_loss)
    losses.append(current_loss)
    content_losses.append(np.mean(content_loss))
    style_losses.append(np.mean(style_loss))

    # Early stopping
    if current_loss < best_loss:
        best_loss = current_loss
        epochs_without_improvement = 0
    else:
        epochs_without_improvement += 1

    if epochs_without_improvement >= early_stopping_patience:
        # if num_epochs > 100: 
        #     print(f"Early stopping at epoch {epoch+1}")
        #     break 
        print(f"Early stopping at epoch {epoch+1}")
        break

    if verbose: 
        if epoch % 10 == 0:
            print(f"Epoch {epoch+1}/{num_epochs}, Batch {i+1}/{len(patches_src)}, "
                f"Content Loss: {content_l.item():.4f}, Style Loss: {style_l.item():.4f}, "
                f"Total Loss: {total_loss.item():.4f}")
            
            # Save the checkpoints
            torch.save(pipeline.state_dict(), f"/root/BiOSGen/checkpoints/pipeline_epoch_{epoch+1}.pth")
            print(f"Model saved at /root/BiOSGen/checkpoints/pipeline_epoch_{epoch+1}.pth")

    # # Visualize the decoded image
    # decoded = decoded.to(torch.uint8)
    # fig, axes = plt.subplots(1, 6, figsize=(20, 15))  # 1x5 grid
    # for i in range(3):
    #     ax = axes[i]
    #     ax.imshow(decoded[0, i].detach().cpu().numpy(), cmap='viridis')
    #     ax.set_title(f'Ch {i}', fontsize=8)
    #     # ax.axis('off')

    # axes[3].imshow(decoded[0].permute(1,2,0).detach().cpu().numpy(), cmap='viridis')
    # axes[3].set_title('Generated patch', fontsize=16)
    # axes[4].imshow(src, cmap='viridis')
    # axes[4].set_title('Original patch', fontsize=16)
    # axes[5].imshow(dst, cmap='viridis')
    # axes[5].set_title('Style patch', fontsize=16)

    # plt.title(f"Epoch {epoch+1}, Loss: {current_loss:.4f}", fontsize=16)

In [ ]:
# After your training loop
# Save model components
torch.save(pipeline.state_dict(), '/root/BiOSGen/checkpoints/pipeline.pth')

# Save the entire model state including optimizer
checkpoint = {
    'epoch': epoch,
    'model_state_dict': pipeline.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    'best_loss': best_loss,
    'losses': losses,
    'content_losses': content_losses,
    'style_losses': style_losses
}

torch.save(checkpoint, '/root/BiOSGen/checkpoints/pipeline_full.pth')

In [ ]:
decoded = decoded.to(torch.uint8)
fig, axes = plt.subplots(1, 6, figsize=(20, 15))  # 1x5 grid
for i in range(3):
    ax = axes[i]
    ax.imshow(decoded[0, i].detach().cpu().numpy(), cmap='viridis')
    ax.set_title(f'Ch {i}', fontsize=8)
    # ax.axis('off')

axes[3].imshow(decoded[0].permute(1,2,0).detach().cpu().numpy(), cmap='viridis')
axes[3].set_title('Generated patch', fontsize=16)
axes[4].imshow(src, cmap='viridis')
axes[4].set_title('Original patch', fontsize=16)
axes[5].imshow(dst, cmap='viridis')
axes[5].set_title('Style patch', fontsize=16)

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(20, 15))  # 1x5 grid

# Plot original source image
axes[0].imshow(np.array(Image.open(tree_src.antibodies[idx_src])), cmap='viridis')
axes[0].set_title(f'Original Tumor {idx_src}', fontsize=16)

# Plot original destination image
axes[1].imshow(np.array(Image.open(tree_dst.antibodies[idx_dst])), cmap='viridis')
axes[1].set_title(f'Style Tumor {idx_dst}', fontsize=16)

# Plot decoded image
gen = []
for i in range(len(patches_src)):
    # Assign idx_src and idx_dst to the patches
    src = patches_src[i]
    dst = patches_dst[i]

    # Convert to tensors
    src_tensor = Utilities.convert_numpy_to_tensor(src).to(device)
    dst_tensor = Utilities.convert_numpy_to_tensor(dst).to(device)

    timesteps = torch.randint(0, 1000, (batch_size,), device=device)  # Random timesteps

    # forward pass
    out = pipeline(src_tensor, dst_tensor, timesteps)
    gen.append(out.to(torch.uint8)[0].permute(1,2,0).detach().cpu().numpy())

generated = PatchesUtilities.replace_patches_in_image(
    original_image=np.array(Image.open(tree_src.antibodies[idx_src])),
    tissue_mask=PatchesUtilities.get_tissue_mask(np.array(Image.open(tree_src.antibodies[idx_src]))),
    generated_patches=gen
)
axes[2].imshow(generated, cmap='viridis')
axes[2].set_title(f'Generated Image', fontsize=16)

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

fig = plt.figure(figsize=(20, 15))
gs = gridspec.GridSpec(2, 2, height_ratios=[1, 1])

# Upper left: Content Loss
ax1 = fig.add_subplot(gs[0, 0])
ax1.plot(content_losses, label='Content Loss')
ax1.set_title('Content Loss', fontsize=15)
ax1.set_xlabel('Epochs')
ax1.set_ylabel('Content Loss')

# Upper right: Style Loss
ax2 = fig.add_subplot(gs[0, 1])
ax2.plot(style_losses, label='Style Loss')
ax2.set_title('Style Loss', fontsize=15)
ax2.set_xlabel('Epochs')
ax2.set_ylabel('Style Loss')

# Bottom (spans both columns): Total Loss
ax3 = fig.add_subplot(gs[1, :])
ax3.plot(losses, label='Total Loss', color='red')
ax3.set_title('Total Loss', fontsize=15)
ax3.set_xlabel('Epochs')
ax3.set_ylabel('Total Loss')

# Add title to fig
fig.suptitle('Losses during Training', fontsize=20)
plt.tight_layout()
plt.show()


In [ ]:
idx_src = 0 # torch.randint(0, len(tree_src.antibodies), (1,)).item()
patches_src = PatchesUtilities.get_image_patches(
    image = np.array(Image.open(tree_src.antibodies[idx_src])),
    tissue_mask = PatchesUtilities.get_tissue_mask(np.array(Image.open(tree_src.antibodies[idx_src]))),
)

idx_dst = 0 # torch.randint(0, len(tree_dst.antibodies), (1,)).item()
patches_dst = PatchesUtilities.get_image_patches(
    image = np.array(Image.open(tree_dst.antibodies[idx_dst])),
    tissue_mask = PatchesUtilities.get_tissue_mask(np.array(Image.open(tree_dst.antibodies[idx_dst]))),
)
print("idx_src: ", idx_src)
print("idx_dst: ", idx_dst)

In [ ]:
torch.randint(0, len(patches_dst), (1,)).item()

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(20, 15))  # 1x5 grid

# Plot original source image
axes[0].imshow(np.array(Image.open(tree_src.antibodies[idx_src])), cmap='viridis')
axes[0].set_title(f'Original Tumor {idx_src}', fontsize=16)

# Plot original destination image
axes[1].imshow(np.array(Image.open(tree_dst.antibodies[idx_dst])), cmap='viridis')
axes[1].set_title(f'Style Tumor {idx_dst}', fontsize=16)

# Plot decoded image
gen = []
for i in range(len(patches_src)):
    # Assign idx_src and idx_dst to the patches
    src = patches_src[i]
    dst = patches_dst[torch.randint(0, len(patches_dst), (1,)).item()]

    # Convert to tensors
    src_tensor = Utilities.convert_numpy_to_tensor(src).to(device)
    dst_tensor = Utilities.convert_numpy_to_tensor(dst).to(device)

    timesteps = torch.randint(0, 1000, (batch_size,), device=device)  # Random timesteps

    # forward pass
    out = pipeline(src_tensor, dst_tensor, timesteps)
    gen.append(out.to(torch.uint8)[0].permute(1,2,0).detach().cpu().numpy())

generated = PatchesUtilities.replace_patches_in_image(
    original_image=np.array(Image.open(tree_src.antibodies[idx_src])),
    tissue_mask=PatchesUtilities.get_tissue_mask(np.array(Image.open(tree_src.antibodies[idx_src]))),
    generated_patches=gen
)
axes[2].imshow(generated, cmap='viridis')
axes[2].set_title(f'Generated Image', fontsize=16)